#Jupyter Notebook for Tweaking the Dataset for the Data Visualization Project

This project will consist of steps to correct the dataset of house prices in Canada. The dataset was obtained from Kaggle and can be found on the following [page](https://www.kaggle.com/datasets/jeremylarcher/canadian-house-prices-for-top-cities).

This process will consist of a few steps, which I will briefly explain below:

- **Step 1: Load the Dataset and Data Cleaning**
  - Check for missing values.
  - Check for outliers.
  - Verify if the values are accurate.
  - Create auxiliary columns.
  - Correct any errors or inconsistencies in the dataset.

- **Step 2: Retrieve better geolocation columns**
  - Split the dataset into 5 parts.
  - Use Google Cloud API to get geolocations based on address.
  - Create auxiliary columns.
  - Merge the parts into one file again


- **Step 3: Generate data for historical analysis** (The dataset was generated in the same day, hence I need to create a fake data column to be able to have time analysis in this dataset)
  - Create different linear regression for each city of this dataset.
  - Generate dates based on their prices and the regression generated using random values.

- **Step 4: Clean Dataset**
  - Delete auxiliary columns created through the process.
  - Keep only the essential columns in the dataset.

*created by: Bruno do Nascimento Beserra*

##Step 0: Import Libraries, and helping functions and class

In [ ]:
# Installing Google Maps API
!pip install -U googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=cfeba347c51b475df32a1764d2b5e5870f1975282687aa4ce1c80cacb8e7fb04
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
# Import libraries

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#from geopy.geocoders import Nominatim      # not used because of reaching fre tier limit too fast
import googlemaps
import glob
from datetime import datetime, timedelta
import random

In [ ]:
#####################################################
################# HELPING FUNCTIONS #################
#####################################################

# Function to remove '#' from some addresses that are giving errors when
# trying to retrieve the real latitude and longitude
def removeHash(dataset):
    if dataset['Address'].str.startswith('#').any():
      dataset['Address'] = dataset['Address'].str.lstrip('#')
    return dataset

# Function to Split the dataset into x files
def splitDataset(dataset, no_parts):
    return np.array_split(dataset, no_parts)

def get_lat_long_city(address, api_key):
    # Initialize the Google Maps client
    gmaps = googlemaps.Client(key=api_key)

    # Geocode the address
    geocode_result = gmaps.geocode(address)

    # Initialize latitude, longitude, and city as None
    lat, lng, city = None, None, None

    # Extract latitude, longitude, and city
    if geocode_result:
        location = geocode_result[0]["geometry"]["location"]
        lat = location["lat"]
        lng = location["lng"]

        # Extract city from address components
        city = None
        for component in geocode_result[0]["address_components"]:
            if "locality" in component["types"]:
                city = component["long_name"]
                break

        return lat, lng, city
    else:
        return None, None, None

def add_dates_to_housing_data(df):
    # Initialize generator
    generator = HousingDateGenerator()

    # Generate dates
    df_with_dates = generator.generate_dates(df)

    return df_with_dates


In [ ]:
###########################################################################################
###################################### HELPING CLASS ######################################
###########################################################################################

class HousingDateGenerator:
    def __init__(self, start_date='2019-01-01', end_date='2024-12-12'):
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.city_params = self._initialize_city_params()

    def _initialize_city_params(self):
        """Initialize base prices and parameters for each city"""
        return {
            # Major Cities - Ontario
            'Toronto': {'base': 1000000, 'growth': 0.12, 'volatility': 0.02},
            'Hamilton': {'base': 650000, 'growth': 0.09, 'volatility': 0.015},
            'Ottawa': {'base': 550000, 'growth': 0.08, 'volatility': 0.015},
            'Kitchener': {'base': 580000, 'growth': 0.085, 'volatility': 0.015},
            'London': {'base': 450000, 'growth': 0.075, 'volatility': 0.015},
            'St. Catharines': {'base': 420000, 'growth': 0.07, 'volatility': 0.015},
            'Oshawa': {'base': 580000, 'growth': 0.085, 'volatility': 0.015},
            'Windsor': {'base': 350000, 'growth': 0.065, 'volatility': 0.015},
            'Barrie': {'base': 500000, 'growth': 0.08, 'volatility': 0.015},
            'Sudbury': {'base': 300000, 'growth': 0.05, 'volatility': 0.01},
            'Kingston': {'base': 400000, 'growth': 0.07, 'volatility': 0.015},
            'Guelph': {'base': 550000, 'growth': 0.08, 'volatility': 0.015},
            'Brantford': {'base': 450000, 'growth': 0.075, 'volatility': 0.015},
            'Thunder Bay': {'base': 250000, 'growth': 0.04, 'volatility': 0.01},
            'Peterborough': {'base': 400000, 'growth': 0.07, 'volatility': 0.015},
            'Sault Ste. Marie': {'base': 200000, 'growth': 0.04, 'volatility': 0.01},
            'Caledon': {'base': 800000, 'growth': 0.095, 'volatility': 0.02},

            # Major Cities - British Columbia
            'Vancouver': {'base': 1200000, 'growth': 0.11, 'volatility': 0.02},
            'Victoria': {'base': 700000, 'growth': 0.09, 'volatility': 0.015},
            'Kelowna': {'base': 600000, 'growth': 0.085, 'volatility': 0.015},
            'Abbotsford': {'base': 650000, 'growth': 0.085, 'volatility': 0.015},
            'Kamloops': {'base': 450000, 'growth': 0.07, 'volatility': 0.015},
            'Nanaimo': {'base': 500000, 'growth': 0.08, 'volatility': 0.015},
            'White Rock': {'base': 800000, 'growth': 0.09, 'volatility': 0.015},
            'Maple Ridge': {'base': 750000, 'growth': 0.085, 'volatility': 0.015},
            'Prince George': {'base': 350000, 'growth': 0.06, 'volatility': 0.01},
            'New Westminster': {'base': 650000, 'growth': 0.085, 'volatility': 0.015},
            'Burnaby': {'base': 900000, 'growth': 0.10, 'volatility': 0.02},

            # Major Cities - Alberta
            'Calgary': {'base': 450000, 'growth': 0.07, 'volatility': 0.01},
            'Edmonton': {'base': 400000, 'growth': 0.06, 'volatility': 0.01},
            'Red Deer': {'base': 350000, 'growth': 0.05, 'volatility': 0.01},
            'Lethbridge': {'base': 300000, 'growth': 0.045, 'volatility': 0.01},
            'Medicine Hat': {'base': 280000, 'growth': 0.04, 'volatility': 0.01},
            'Airdrie': {'base': 400000, 'growth': 0.065, 'volatility': 0.01},

            # Major Cities - Quebec
            'Montreal': {'base': 500000, 'growth': 0.09, 'volatility': 0.015},
            'Quebec': {'base': 300000, 'growth': 0.07, 'volatility': 0.015},
            'Sherbrooke': {'base': 250000, 'growth': 0.06, 'volatility': 0.01},
            'Trois-Rivieres': {'base': 200000, 'growth': 0.055, 'volatility': 0.01},

            # Major Cities - Manitoba/Saskatchewan
            'Winnipeg': {'base': 350000, 'growth': 0.05, 'volatility': 0.01},
            'Saskatoon': {'base': 330000, 'growth': 0.045, 'volatility': 0.01},
            'Regina': {'base': 320000, 'growth': 0.04, 'volatility': 0.01},

            # Major Cities - Atlantic Canada
            'Halifax': {'base': 400000, 'growth': 0.08, 'volatility': 0.015},
            'Moncton': {'base': 250000, 'growth': 0.07, 'volatility': 0.015},
            'Saint John': {'base': 200000, 'growth': 0.06, 'volatility': 0.01},
            "St. John's": {'base': 300000, 'growth': 0.04, 'volatility': 0.01}
            # Add more cities as needed with their parameters
        }

    def _calculate_expected_price(self, city, date):

        years_since_start = (pd.to_datetime(date) - self.start_date).days / 365.25
        expected_price = params['base'] * (1 + params['growth']) ** years_since_start

        # Create seasonal variation (as prices tend to be higher during summer)
        month = pd.to_datetime(date).month
        seasonal_factor = 1 + 0.05 * np.sin((month - 6) * np.pi / 6)

        # Add market cooling effect
        if pd.to_datetime(date).year == 2023:
            expected_price *= 0.95

        return expected_price * seasonal_factor

    def generate_dates(self, df):

        df = df.copy()
        dates = []

        for _, row in df.iterrows():
            city = row['City']
            price = row['Price']

            # Generate candidate dates and find best match
            candidate_dates = pd.date_range(self.start_date, self.end_date, freq='D')
            expected_prices = [self._calculate_expected_price(city, date) for date in candidate_dates]

            # Add some randomness to price matching to avoid clustering
            noise = np.random.normal(0, 0.1, len(expected_prices))
            price_diff = np.abs(np.array(expected_prices) * (1 + noise) - price)

            # Get the index of the closest match with some randomness
            best_matches = np.argsort(price_diff)[:10]  # Get top 10 matches
            chosen_index = np.random.choice(best_matches)  # Randomly select from top matches

            dates.append(candidate_dates[chosen_index])

        df['Date'] = dates
        return df

##Step 1: Load the Dataset and Data Cleaning


In [ ]:
# Load the dataset
dataset = pd.read_csv("/content/canada-houses.csv", index_col=0, sep=",", decimal=".", encoding="ISO-8859-1")

# Removing Hashtags
dataset = removeHash(data)

# Reset index
dataset = dataset.reset_index()

# Create column with whole address to pass to the API function
dataset['Complete_Address'] = dataset['Address'] + ', ' + dataset['City'] + ', ' + dataset['Province'] + ', Canada'

In [ ]:
#Checking data post processing
dataset.head()

,City,Price,Address,Number_Beds,Number_Baths,Province,Population,Latitude,Longitude,Median_Family_Income,Complete_Address,Lat,Long,new_city
0,Toronto,779900.0,318 -20 SOUTHPORT ST,3,2,Ontario,5647656,43.7417,-79.3733,97000.0,"318 -20 SOUTHPORT ST, Toronto, Ontario, Canada",43.637299,-79.472296,Toronto
1,Toronto,799999.0,818 -60 SOUTHPORT ST,3,1,Ontario,5647656,43.7417,-79.3733,97000.0,"818 -60 SOUTHPORT ST, Toronto, Ontario, Canada",43.638758,-79.473860,Toronto
2,Toronto,799900.0,714 -859 THE QUEENSWAY,2,2,Ontario,5647656,43.7417,-79.3733,97000.0,"714 -859 THE QUEENSWAY, Toronto, Ontario, Canada",43.624558,-79.509776,Toronto
3,Toronto,1200000.0,275 MORTIMER AVE,4,2,Ontario,5647656,43.7417,-79.3733,97000.0,"275 MORTIMER AVE, Toronto, Ontario, Canada",43.686590,-79.343463,Toronto
4,Toronto,668800.0,420 -388 RICHMOND ST,1,1,Ontario,5647656,43.7417,-79.3733,97000.0,"420 -388 RICHMOND ST, Toronto, Ontario, Canada",43.648196,-79.395030,Toronto


In [ ]:
#Checking data post processing
dataset.tail()

,City,Price,Address,Number_Beds,Number_Baths,Province,Population,Latitude,Longitude,Median_Family_Income,Complete_Address,Lat,Long,new_city
35763,Halifax,439900.0,301 60 Walter Havill Drive,2,2,Nova Scotia,431479,44.8857,63.1005,86753.0,"301 60 Walter Havill Drive, Halifax, Nova Scot...",44.632016,-63.633648,Halifax
35764,Halifax,6990000.0,1160 Rockcliffe Street,5,5,Nova Scotia,431479,44.8857,63.1005,86753.0,"1160 Rockcliffe Street, Halifax, Nova Scotia, ...",44.632911,-63.595990,Halifax
35765,Halifax,419900.0,212 60 Walter Havill Drive,2,2,Nova Scotia,431479,44.8857,63.1005,86753.0,"212 60 Walter Havill Drive, Halifax, Nova Scot...",44.632016,-63.633648,Halifax
35766,Halifax,949900.0,10 Idlewylde Road,3,1,Nova Scotia,431479,44.8857,63.1005,86753.0,"10 Idlewylde Road, Halifax, Nova Scotia, Canada",44.633924,-63.621585,Halifax
35767,Halifax,592900.0,208 2842-2856 Gottingen,2,1,Nova Scotia,431479,44.8857,63.1005,86753.0,"208 2842-2856 Gottingen, Halifax, Nova Scotia,...",44.660694,-63.596161,Halifax


In [ ]:
dataset.shape

(35768, 14)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35768 entries, 0 to 35767
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City                  35768 non-null  object 
 1   Price                 35768 non-null  float64
 2   Address               35768 non-null  object 
 3   Number_Beds           35768 non-null  int64  
 4   Number_Baths          35768 non-null  int64  
 5   Province              35768 non-null  object 
 6   Population            35768 non-null  int64  
 7   Latitude              35768 non-null  float64
 8   Longitude             35768 non-null  float64
 9   Median_Family_Income  35768 non-null  float64
 10  Complete_Address      35768 non-null  object 
 11  Lat                   35764 non-null  float64
 12  Long                  35764 non-null  float64
 13  new_city              35329 non-null  object 
dtypes: float64(6), int64(3), object(5)
memory usage: 3.8+ MB


##Step 2: Retrieve better geolocation columns


In [ ]:
# Split the DataFrame into 5 parts
split_dataframes = splitDataset(dataset, 2)

folder_path = '/content/drive/MyDrive/Estudos/auxDouglas/'
# Displaying the first few rows of each part
for i, part in enumerate(split_dataframes, 1):
    file_name = f'part{i}.csv'
    part.to_csv(f'{folder_path}part{i}.csv', index=False)
    print(f"Part {i}:\n{part.head()}\n")

Part 1:
              City      Price                     Address  Number_Beds  \
0  New Westminster  1700000.0           7566 144TH STREET            8   
1  New Westminster   688800.0  1709-4730 LOUGHEED HIGHWAY            1   
2  New Westminster  1270000.0           10013 159A STREET            3   
3  New Westminster   640000.0    604-13655 FRASER HIGHWAY            2   
4  New Westminster   409900.0        304-15268 100 AVENUE            1   

   Number_Baths          Province  Population  Latitude  Longitude  \
0             6  British Columbia       78916   49.2069  -122.9111   
1             1  British Columbia       78916   49.2069  -122.9111   
2             2  British Columbia       78916   49.2069  -122.9111   
3             2  British Columbia       78916   49.2069  -122.9111   
4             1  British Columbia       78916   49.2069  -122.9111   

   Median_Family_Income  
0               82000.0  
1               82000.0  
2               82000.0  
3               82000.

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
# Geo-coding key
geo_coding_key = {google_api_key_generated}

In [ ]:
# Apply the get_lat_long function to each address in the DataFrame
dataset[['Lat', 'Long', 'new_city']] = dataset['Complete_Address'].apply(lambda x: pd.Series(get_lat_long_city(x, geo_coding_key)))

      City      Price                   Address  Number_Beds  Number_Baths  \
0  Burnaby  3195000.0        2002 W 35TH AVENUE            5             2   
1  Burnaby   949900.0  312-626 ALEXANDER STREET            2             3   
2  Burnaby  8900000.0    701-277 THURLOW STREET            3             4   
3  Burnaby   999900.0    2825 ST ANDREWS STREET            3             3   
4  Burnaby   725000.0  216-223 MOUNTAIN HIGHWAY            2             2   

           Province  Population  Latitude  Longitude  Median_Family_Income  \
0  British Columbia      249125   49.2667  -122.9667               83000.0   
1  British Columbia      249125   49.2667  -122.9667               83000.0   
2  British Columbia      249125   49.2667  -122.9667               83000.0   
3  British Columbia      249125   49.2667  -122.9667               83000.0   
4  British Columbia      249125   49.2667  -122.9667               83000.0   

                                    Complete_Address        La

In [ ]:
# Folder Location
folder_path = '/content/drive/MyDrive/Estudos/auxDouglas/geo_corrected_real/'

#Concatenate all parts from dataset corrected
all_files = glob.glob(f"{folder_path}/part*.csv")
dataset = pd.concat((pd.read_csv(file) for file in all_files), ignore_index=True)

# Save the concatenated dataset into the same folder
dataset.to_csv(f'{folder_path}real_wholeDataset.csv', index=False)

##Step 3: Generate data for historical analysis

In [ ]:
# Get unique values from the city column to know which cities appear in our dataset
# This info was used to ask an AI to create random linear regressions for these cities
unique_values = data['City'].unique()

print(unique_values)

['Toronto' 'Vancouver' 'Calgary' 'Edmonton' 'Ottawa' 'Winnipeg' 'Hamilton'
 'Kitchener' 'London' 'St. Catharines' 'Oshawa' 'Victoria' 'Windsor'
 'Saskatoon' 'Regina' "St. John's" 'Barrie' 'Kelowna' 'Abbotsford'
 'Sudbury' 'Kingston' 'Guelph' 'Moncton' 'Brantford' 'Thunder Bay'
 'Saint John' 'Peterborough' 'Red Deer' 'Kamloops' 'Nanaimo' 'Lethbridge'
 'White Rock' 'Maple Ridge' 'Prince George' 'Sault Ste. Marie'
 'Medicine Hat' 'New Westminster' 'Caledon' 'Burnaby' 'Airdrie' 'Montreal'
 'Sherbrooke' 'Quebec' 'Trois-Rivieres' 'Halifax']


In [ ]:
# Add dates to the dataset
data_with_dates = add_dates_to_housing_data(data)

# Additional analysis
print("\nDate distribution by city:")
print(data_with_dates.groupby('City')['Date'].agg(['min', 'max', 'count']))


Date distribution by city:
                        min        max  count
City                                         
Abbotsford       2019-01-02 2024-12-10    755
Airdrie          2019-01-24 2024-12-12    225
Barrie           2019-01-01 2024-12-12   1012
Brantford        2019-01-07 2024-12-12    628
Burnaby          2019-01-01 2024-12-10   1334
Caledon          2019-01-01 2024-12-12   1336
Calgary          2019-01-01 2024-12-12   1322
Edmonton         2019-01-02 2024-12-12   1351
Guelph           2019-01-01 2024-12-12    436
Halifax          2019-01-13 2024-11-26     91
Hamilton         2019-01-02 2024-12-12   1289
Kamloops         2019-01-01 2024-12-11    442
Kelowna          2019-01-01 2024-12-12   1280
Kingston         2019-01-04 2024-12-11    486
Kitchener        2019-01-01 2024-12-12   1199
Lethbridge       2019-01-05 2024-12-11    379
London           2019-01-01 2024-12-12   1345
Maple Ridge      2019-01-02 2024-12-12   1282
Medicine Hat     2019-01-13 2024-12-08    277
Moncto

##Step 4: Clean Dataset

In [ ]:
# Read the file
data = pd.read_csv("/content/drive/MyDrive/Estudos/auxDouglas/geo_corrected_real/real_wholeDataset.csv")

# Remove aux_column created
data = data.drop('Complete_Address', axis=1)

# Rename Columns for attribute names that make more sense
data = data.rename(columns={'Latitude': 'fake_Latitude', 'Longitude': 'fake_Longitude', 'Lat': 'Latitude', 'Long': 'Longitude'})


In [ ]:
# Check dataset afterwards
data.head()

,City,Price,Address,Number_Beds,Number_Baths,Province,Population,fake_Latitude,fake_Longitude,Median_Family_Income,Latitude,Longitude,new_city,Date
19599,Kelowna,299900.0,125 1860 Boucherie Road,3,2,British Columbia,181380,49.8881,-119.4956,85000.0,49.832903,-119.592482,NaN,2019-01-01
121,Toronto,765000.0,1802 -75 QUEENS WHARF RD,2,1,Ontario,5647656,43.7417,-79.3733,97000.0,43.638479,-79.398085,Toronto,2019-01-01
1967,Vancouver,895000.0,706-933 HORNBY STREET,2,2,British Columbia,2426160,49.2500,-123.1000,90000.0,49.281325,-123.124051,Vancouver,2019-01-01
12944,Oshawa,359900.0,1201 -55 WILLIAM ST E,1,1,Ontario,335949,43.9000,-78.8500,102000.0,43.900053,-78.862297,Oshawa,2019-01-01
17044,St. John's,179900.0,10 Nash Crescent,2,1,Newfoundland and Labrador,185565,47.4817,-52.7971,85000.0,47.527040,-52.801276,Mount Pearl,2019-01-01


##Dump Part (things that were used to understand the problem, but not used in the actual process, this feature worked at first but reached the free tier with not many inquiries and became useless, but it still works, for few queries)

In [ ]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapiExercises")
# Provide the address
address = "318 -20 SOUTHPORT ST"

# Get the location
location = geolocator.geocode(address)

# Check if the location was found and print latitude and longitude
if location:
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
else:
    print("Location not found!")